# Testing the ShuffleService

In [7]:
import hashlib
from arcanum.shuffle import ShuffleService

In [8]:
# Test consistency (should be same)
seed1 = ShuffleService.generate_seed("Career guidance", 5, include_date=False)
seed2 = ShuffleService.generate_seed("Career guidance", 5, include_date=False)
print(f"Consistent seeds match: {seed1 == seed2}")

Consistent seeds match: True


In [9]:
# Test date-based (should be different tomorrow)
daily1 = ShuffleService.generate_seed("Daily draw", 3, include_date=True)
print(f"Today's daily seed: {daily1}")

# Simulate what would happen with same question but different date
question = "Daily draw"
shuffle_count = 3
fake_date = "2025-07-20"  # Tomorrow
combined = f"{question}|{shuffle_count}|{fake_date}"
fake_hash = hashlib.sha256(combined.encode("utf-8"))
fake_seed = int(fake_hash.hexdigest()[:8], 16)
print(f"Tomorrow's seed would be: {fake_seed}")
print(f"Seeds are different: {daily1 != fake_seed}")

Today's daily seed: 2598452311
Tomorrow's seed would be: 3606867473
Seeds are different: True


## Testing DrawingService

In [10]:
from arcanum.deck import Deck
from arcanum.drawing import DrawingService

In [11]:
# Set up
deck = Deck()
cards = deck.get_cards()
seed = ShuffleService.generate_seed("Test reading", 5)
shuffled = ShuffleService.shuffle_cards(cards, seed)

In [12]:
# Draw some cards
drawer = DrawingService(reversal_chance=0.5)  # 50% reversal for testing
drawn = drawer.draw_cards(shuffled, 3, ["Past", "Present", "Future"])

In [13]:
for card in drawn:
    print(card)

Past: 4 of Swords (Reversed)
Present: Ace of Pentacles
Future: The Emperor (Reversed)


In [14]:
from arcanum.spreads import SingleCardSpread, ThreeCardSpread, CelticCrossSpread


In [15]:
# Try different layouts
single = SingleCardSpread()
three = ThreeCardSpread()
celtic = CelticCrossSpread()

In [16]:
print(f"{single.name}: {single.card_count} cards")
for pos in single.positions:
    print(f"  {pos.name}: {pos.description}")

print(f"\n{three.name}: {three.card_count} cards")
for pos in three.positions:
    print(f"  {pos.name}: {pos.description}")

Single Card: 1 cards
  Focus: The main energy or message for you

Past, Present, Future: 3 cards
  Past: Influences from your past affecting the situation
  Present: Current energies and circumstances
  Future: Likely outcome or direction


## Testing Reading

In [17]:
from arcanum.reading import ReadingService

In [18]:
# Create reading service
reader = ReadingService(reversal_chance=0.5)

In [19]:
# Perform a reading
reading = reader.perform_reading(
    question="What should I focus on this week?",
    spread=three,
    shuffle_count=5
)

print(reading)
print(f"\nSeed used: {reading.seed}")

Question: What should I focus on this week?
Spread: Past, Present, Future
Date: 2025-07-19 16:46

Past: 9 of Pentacles
Present: 5 of Cups (Reversed)
Future: The Hierophant

Seed used: 1014792876


In [20]:
# Same question should give same result
reading2 = reader.perform_reading(
    question="What should I focus on this week?",
    spread=three,
    shuffle_count=5
)

In [21]:
print(f"Readings are identical: {reading.cards[0].card.name == reading2.cards[0].card.name}")

Readings are identical: True
